In [ ]:
import glob
import os
import numpy as np
from nbodykit.lab import cosmology
import pandas as pd
import matplotlib.pyplot as plt
import copy
from scipy.interpolate import InterpolatedUnivariateSpline
from solver import KmouExpansionJordan

In [ ]:
plt.style.use('../kp-jc-6-paper-2/notebooks/my_style_new.ini')

In [ ]:
def pk_to_file(Om, Ob, h,  ns, As=None, sigma8=None, z=0, k_vals=np.logspace(-4,2, 1000), mode="out", bs_fact=1, out_path='./'):
    if As is not None:
        nb_cosmo = cosmology.Cosmology(h=h, Omega0_b=Ob, 
                            Omega0_cdm=Om - Ob, n_s=ns, m_ncdm=[], A_s=As)
    else:
        try:
            nb_cosmo = cosmology.Cosmology(h=h, Omega0_b=Ob, 
                            Omega0_cdm=Om - Ob, n_s=ns, m_ncdm=[]).match(sigma8)
            As = nb_cosmo.A_s
        except:
            raise Exception('Need to specify at least one between As and sigma8')
    
    Plin = cosmology.LinearPower(nb_cosmo, redshift=z, transfer='CLASS')
    
    df_CLASS = pd.DataFrame(index=k_vals, data= Plin(k_vals))
    if mode=='local':
        return df_CLASS
    elif mode=='cosmo':
        return nb_cosmo
    elif mode=='out':
        out_folder = f'./{Params_path}/outPk/Om{Om:.4}_ns{ns:.4}_As{As:.4}_h{h:.4}_Ob{Ob:.4}/'
        if not os.path.exists(out_folder):
            os.makedirs(out_folder)
        df_CLASS.to_csv(f'{out_folder}/input_power_spectrum_CLASS.dat',sep='\t', header=False, index=True,  encoding='utf-8')
    elif mode=='backscale_out':
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        (df_CLASS*bs_fact).to_csv(f'{out_path}/input_power_spectrum.dat',sep='\t', header=False, index=True,  encoding='utf-8')

In [ ]:
def get_param(param, string):
    return string.split(param)[-1].split('_')[0]

### Folder structure

In [ ]:
sim_root = 'BensProjectSims_Kmou'

In [ ]:
sim_folder=f'/Users/fiorini/GitHub/K-mouflage/Data/{sim_root}'
sim_names=['LCDM']
beta_list=[0.125, 0.2]
K0_list=[1]
box_sizes=[400,]# 400, 400, 400, 400, 400]
Nparts=[512,]# 512, 512, 512, 512, 512*2]
Nmeshs=[512*3,]# 512*3, 512*3, 512*3, 512*6, 512*6]
Nsteps=[40,]# 60, 80, 100, 100, 100]
Seeds=[73119]
ReversePhases = False
Params_path=f'Params_for_{sim_root}'
in_FML_params = f'{Params_path}/parameterfile.lua'
z_ini = 49

In [ ]:
# cosmological parameters for Kmou
h_list=[ 0.6774] #, 0.6774] # 0.67
Omega0_b_list=[0.0486] #, 0.0486] # 0.049
Omega0_m_list=[0.3089] #, 0.3089] # 0.319
n_s_list=[0.9667] #, 0.9667] # 0.96
A_s_list = [2.06535e-9] #, 2.066e-9] # 2.1e-9

In [ ]:
# Save power spectrem to file # acutally now used only for the folder
for h, Ob, Om, ns, As in zip(h_list, Omega0_b_list, Omega0_m_list, n_s_list, A_s_list):
    pk_to_file(Om, Ob, h, ns, As=As) 
    # nb_cosmo = pk_to_file(Om, Ob, h, ns, As=As, mode='cosmo') 

In [ ]:
for beta in beta_list:
    for K0 in K0_list:
        sim_names.append(f'Kmou_beta{beta}_K{K0}')

In [ ]:
sim_names

### Cosmologies

In [ ]:
input_Pk_folder = f'./{Params_path}/outPk/'
comsos_files = glob.glob(f'{input_Pk_folder}/*/')

cosmos = [file.split('/')[-2] for file in comsos_files]

In [ ]:
# rm ./Params_for_KPJC6_Paper1/outPk/ -r

In [ ]:
cosmos

# Models

In [ ]:
sim_names

In [ ]:
mod_message = 'Line modified by python'

### Read default run and parameter file

In [ ]:
with open(in_FML_params, 'r') as file:
    # read default parameters
    default_FML_params_data = file.readlines()

In [ ]:
default_FML_params_data[24] = 'output_particles = false\n'

In [ ]:
default_FML_params_data[240] = f'ic_fix_amplitude = true -- {mod_message} \n'

In [ ]:
# Wanted?
default_FML_params_data[61] = f'cosmology_TCMB_kelvin = 2.7255e-7 -- {mod_message} \n'

### Create background and force files

In [ ]:
table_file = './Data/Output/n2-beta0p2-k1/out_table.txt'
a_E, a_J, E_J, dE_J_over_da, phi_J, dphi_J_over_da, G_eff_over_G, A_conf, D_kmou = np.loadtxt(table_file).T

In [ ]:
# Convert redshifts between frames
zJ_from_zE = lambda zE, dum_exp: 1/dum_exp.get_a_Jor(1/(zE+1)) -1  

In [ ]:
def write_data_flex(data, output_filename_as_string, datanames=''):
    '''-- Copied from HiCOLA --'''
    datafile_id = open(output_filename_as_string, 'wb')    #here you open the ascii file
    format_list = list(np.repeat('%.4e',len(data)))
    newdata = []
    for i in data:
        newdata.append(np.array(i[::]))
    realdata = np.array(newdata)
    realdata = realdata.T     #here you transpose your data, so to have it in two columns
    np.savetxt(datafile_id, realdata, fmt=format_list, header=datanames)    #here the ascii file is populated.
    datafile_id.close()    #close the file

def get_background(path, dum_exp):
    _, table = dum_exp.eval()
    a, phi, _, E, E_a, A, mu_kmou = table
    # Unpack useful quantities
    a_arr = a[::-1]
    E_arr = E[::-1]
    UE_prime_UE_arr = (a*E_a/(E))[::-1]
    coupling_factor_arr = mu_kmou[::-1]-1
    chioverdelta_arr = np.zeros_like(a_J)
    GG4_arr = A[::-1]**2
    force_data =[a_arr,chioverdelta_arr,coupling_factor_arr, GG4_arr]

    # Write data to files    
    filename_expansion = f'/{path}/files/expansion.txt'
    filename_force = f'/{path}/files/force.txt'
    write_data_flex([a_arr,E_arr, UE_prime_UE_arr], filename_expansion)
    write_data_flex(force_data, filename_force)

### Compare with old approach

In [ ]:
dum_exp = KmouExpansionJordan(lamb_val=1.476)
dum_exp.solve()
header, table = dum_exp.eval()

In [ ]:
df = pd.DataFrame(data=table.T, columns=header)
df

In [ ]:
df.plot(y='mu_kmou', x='a', logx=True)
plt.plot(a_J, G_eff_over_G, '--')

### Let's go

In [ ]:
test = False

In [ ]:
ReversePhases = False

In [ ]:
out_z_list = [2, 1, 0.5, 0] # not used

In [ ]:
Boxes = range(1,len(Seeds)+1)
if ReversePhases==True:
    Boxes=[1]
for sim_name in sim_names:
#     for  in box_sizes:
    for box_size,Npart,Nmesh, Ns in zip(box_sizes,Nparts,Nmeshs, Nsteps):
#             for Nmesh in Nmeshs:
        for Box in Boxes:
            Seed = Seeds[Box-1]
            for cosmo in cosmos:
                if ReversePhases:
                    BoxFolder = f'Box{Box}_Reverse'
                else:
                    BoxFolder = f'Box{Box}'
                # create directory tree
                path = f'{sim_folder}/L{box_size}_NP{Npart}_NM{Nmesh}_NS{Ns}/{sim_name}/{cosmo}/{BoxFolder}/'
                if not os.path.exists(path):
                    os.makedirs(path)
                # create log folders
                log_path = f'{path}/logs/'
                if not os.path.exists(log_path):
                    os.makedirs(log_path)
                # copy default FML params in sim's folder
                out_FML_params = f'{path}/parameterfile.lua'
                os.system(f'cp {in_FML_params} {out_FML_params}') 

                #==== customize paramter file with cosmology params and MG params ====#
                FML_params_data = default_FML_params_data.copy()

                ns = get_param('ns', cosmo)
                As = get_param('As', cosmo)
                Om = float(get_param('Om', cosmo))
                Ob = float(get_param('Ob', cosmo))
                h = get_param('h', cosmo)
                Ol = 1-Om
                Ocdm = np.round(Om - Ob,5)
                
                if sim_name=='LCDM':
                    dum_exp = KmouExpansionJordan(lamb_val=10, Om0=Om)
                    dum_exp.solve()
                    dum_exp.get_growth_LCDM()
                    D_bs = (dum_exp.D_LCDM.sol(1/z_ini)[1] /dum_exp.D_LCDM.sol(1)[1])
                else:
                    beta = float(get_param('beta', sim_name))
                    K0 = float(get_param('K', sim_name))
                    dum_exp = KmouExpansionJordan(lamb_val=10, beta=beta, K0=K0, Om0=Om)
                    dum_exp.tune_lambda(maxiter=10)
                    dum_exp.get_growth()
                    dum_exp.get_growth_LCDM()
                    D_bs = (dum_exp.D_kmou_J.sol(1/z_ini)[1] /dum_exp.D_LCDM.sol(1)[1])
                    D_bs_LCDM = (dum_exp.D_LCDM.sol(1/z_ini)[1] /dum_exp.D_LCDM.sol(1)[1])
                    print(f'The ratio of bs_fact with the LCDM case is ={(D_bs/D_bs_LCDM)**2}.')
#                 print(f'Backscaling to z={z_ini} with D_bs={D_bs}, i.e. using bs_fact={D_bs**2}.')
                    
                # copy input power spectrum in sim's folder
                files_dir= f'{path}files/'
                if not os.path.exists(files_dir):
                    os.makedirs(files_dir)
                pk_to_file(Om, Ob, h, ns, As, z=0, mode='backscale_out', bs_fact=D_bs**2, out_path=files_dir)
                
                # Update Front End dictionary
                if sim_name=='LCDM':
                    FML_params_data[49] = 'cosmology_model = "LCDM"\n'
                # change relevant lines
                FML_params_data[19] = f'particle_Npart_1D = {Npart} -- {mod_message} \n'
                FML_params_data[20] = f'force_nmesh = {Nmesh} -- {mod_message} \n'
                FML_params_data[21] = f'ic_random_seed = {Seed} -- {mod_message} \n'
                FML_params_data[22] = 'output_redshifts = {'+', '.join([str(z) for z in out_z_list])+'}'+f' -- {mod_message} \n'
#                 FML_params_data[22] = 'output_redshifts = {'+', '.join([str(np.round(zJ_from_zE(np.array(z),dum_exp),3)) for z in out_z_list])+'}'+f' -- {mod_message} \n'
                FML_params_data[23] = 'timestep_nsteps = {'+str(Ns)+'}'+f' -- {mod_message} \n' #
                FML_params_data[34] = f'simulation_boxsize = {box_size} -- {mod_message} \n'
                FML_params_data[51] = f'cosmology_OmegaCDM = {Ocdm} -- {mod_message} \n'
                FML_params_data[53] = f'cosmology_Omegab = {Ob} -- {mod_message} \n'
                FML_params_data[57] = f'cosmology_OmegaLambda = {Ol} -- {mod_message} \n'
                FML_params_data[63] =f'cosmology_h = {h} -- {mod_message} \n'
                FML_params_data[65] = f'cosmology_As = {As} -- {mod_message} \n'
                FML_params_data[67] = f'cosmology_ns = {ns} -- {mod_message} \n'
                FML_params_data[261] = f'ic_input_redshift = {z_ini} -- {mod_message} \n'
                if ReversePhases:
                    FML_params_data[242] = f'ic_reverse_phases = true -- {mod_message} \n'
                    
                # set gravity model for force
                if sim_name=='CuGal' or sim_name=='Kmou':
                    FML_params_data[100] = f'gravity_model = "HiCOLA" -- {mod_message} \n'
                else:
                    FML_params_data[100] = f'gravity_model = "GR" -- {mod_message} \n'

                with open(out_FML_params, 'w') as f:
                    f.writelines( FML_params_data )
                    
                # Write background and force files
                get_background( path, dum_exp)

                # if not os.path.exists(f'{path}/particles/pofk_snap_cb_z0.000.txt'):
                print(path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
table_1 = np.loadtxt('/Users/fiorini/GitHub/K-mouflage/Data//BensProjectSims_Kmou/L400_NP512_NM1536_NS40/Kmou_beta0.125_K1/Om0.3089_ns0.9667_As2.065e-09_h0.6774_Ob0.0486/Box1/files/force.txt')
table_2 = np.loadtxt('/Users/fiorini/GitHub/K-mouflage/Data//BensProjectSims_Kmou/L400_NP512_NM1536_NS40/Kmou_beta0.2_K1/Om0.3089_ns0.9667_As2.065e-09_h0.6774_Ob0.0486/Box1/files/force.txt')
plt.plot(table_1[:,0], table_1[:,2])
plt.plot(table_2[:,0], table_2[:,2], '--')
plt.semilogx()